In [ ]:
### JUPYTER NOTEBOOK example ###
import os
import sys
import numpy as np
import nibabel as nib

from dbsi_toolbox.model import DBSI_Fused 
from dbsi_toolbox.utils.tools import load_data

print("Libraries loaded successfully.")


DATA_DIR = 'Your/Data/Directory/Here'  # <-- UPDATE THIS PATH

f_dwi  = os.path.join(DATA_DIR, 'DWI_Preprocessed_Image.nii.gz') # <-- UPDATE THIS PATH
f_bval = os.path.join(DATA_DIR, 'DWI_bvalues.bval') # <-- UPDATE THIS PATH
f_bvec = os.path.join(DATA_DIR, 'DWI_bvecs.bvec') # <-- UPDATE THIS PATH
f_mask = os.path.join(DATA_DIR, 'DWI_Preprocessed_Image_brain_mask.nii.gz') # <-- UPDATE THIS PATH
output_dir = os.path.join(DATA_DIR, 'Your/Output/Directory/Here')  # <-- UPDATE THIS PATH

os.makedirs(output_dir, exist_ok=True)
print(f"Output directory ready: {output_dir}")

print(">>> Loading Data...")
try:
    data, affine, bvals, bvecs, mask = load_data(f_dwi, f_bval, f_bvec, f_mask, verbose=True)
    print(f"Data loaded. Volume shape: {data.shape}")
except Exception as e:
    print(f"Error: {e}")

print("\n>>> Starting DBSI Fusion Pipeline...")

# Initialize model
model = DBSI_Fused(enable_step2=True)

results = model.fit(data, bvals, bvecs, mask, run_calibration=True)

print(">>> Analysis Complete.")


# Cell 6: Save to Disk
print(f"\n>>> Saving maps to: {output_dir}")

map_names = [
    'fiber_fraction', 
    'restricted_fraction', 
    'hindered_fraction', 
    'water_fraction', 
    'axial_diffusivity', 
    'radial_diffusivity',
    'fiber_fa_intrinsic', 
    'mean_iso_adc',
    'ad_linear',
    'rd_linear'
]

for i, name in enumerate(map_names):
    out_img = nib.Nifti1Image(results[..., i], affine)
    fname = os.path.join(output_dir, f'dbsi_{name}.nii.gz')
    nib.save(out_img, fname)
    print(f"Saved: {fname}")

print("\nAll files saved successfully.")

